In [ ]:
!pip install accelerate -U
!pip install sentence-transformers

import pandas as pd
from sentence_transformers import SentenceTransformer, LoggingHandler , models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader
import logging
from huggingface_hub import login , HfApi
import pandas as pd

In [ ]:
df = pd.read_csv("/kaggle/input/sentiment/sentiment task - Sheet1.csv")
df.columns = ['text' , "label"]

# Create pairs dataset

In [ ]:
from itertools import combinations
pair_data = {'text1': [], 'text2': [], 'label': []}
unique_labels = df['label'].unique()

for label in unique_labels:
    samples_with_label = df[df['label'] == label]
    pairs_same_class = list(combinations(samples_with_label['text'], 2))
    pair_data['text1'].extend([pair[0] for pair in pairs_same_class])
    pair_data['text2'].extend([pair[1] for pair in pairs_same_class])
    pair_data['label'].extend([1] * len(pairs_same_class))

for label1, label2 in combinations(unique_labels, 2):
    samples_with_label1 = df[df['label'] == label1]
    samples_with_label2 = df[df['label'] == label2]
    pairs_diff_class = [(text1, text2) for text1 in samples_with_label1['text'] for text2 in samples_with_label2['text']]
    pair_data['text1'].extend([pair[0] for pair in pairs_diff_class])
    pair_data['text2'].extend([pair[1] for pair in pairs_diff_class])
    pair_data['label'].extend([-1] * len(pairs_diff_class))
    
new_df = pd.DataFrame(pair_data)
new_df = new_df.sample(frac = 1)
new_df.reset_index(drop = True , inplace = True)
new_df.label = new_df.label.astype(float)

In [ ]:
new_df.label.value_counts()

In [ ]:
# only take a sample
# dataset = pd.concat((new_df[lambda x : x.label == 1.0].sample(619283),new_df[lambda x : x.label == -1.0].sample(619283)))
# del new_df
# import gc
# gc.collect()

In [ ]:
class LoggingCS(losses.CosineSimilarityLoss):
    def forward(self, sentence_features, labels):
        loss_value = super().forward(sentence_features, labels)
        _losses.append(loss_value.item())
        return loss_value
_losses = []

In [ ]:
from tqdm.notebook import tqdm
from sentence_transformers.util import cos_sim
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util , InputExample
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, LoggingHandler , losses, models

similarity_dataset = new_df

train_samples = []

for row in tqdm(similarity_dataset.itertuples()):
    train_samples.append(InputExample(texts=[row[1] , row[2]], label=row[3]))
    
from torch.utils.data import DataLoader
batch_size = 230

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=batch_size)
model = SentenceTransformer("" )
model._first_module().max_seq_length = 50

train_loss = LoggingCS(model=model)
epochs = 1
lr = 2e-5
model.fit( train_objectives = [(train_dataloader , train_loss)],
          epochs=epochs, 
          output_path='output_of_bert/' ,optimizer_params={'lr': lr} )

In [ ]:
import matplotlib.pyplot as plt
plt.plot(_losses)

In [ ]:
login("")

In [ ]:
from huggingface_hub import HfApi ,login
model.save("my_model")
api = HfApi()

api.upload_folder(
    repo_id="",
    folder_path="",
    repo_type="model",
)